Now we will continue on the [Conversation AI](https://conversationai.github.io/) dataset seen in [week 4 homework and lab](https://github.com/MIDS-scaling-up/v2/tree/master/week04). 
We shall use a version of pytorch BERT for classifying comments found at [https://github.com/huggingface/pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT).  

The original implementation of BERT is optimised for TPU. Google released some amazing performance improvements on TPU over GPU, for example, see [here](https://medium.com/@ranko.mosic/googles-bert-nlp-5b2bb1236d78) - *BERT relies on massive compute for pre-training ( 4 days on 4 to 16 Cloud TPUs; pre-training on 8 GPUs would take 40–70 days).*. In response, Nvidia released [apex](https://devblogs.nvidia.com/apex-pytorch-easy-mixed-precision-training/), which gave mixed precision training. Weights are stored in float32 format, but calculations, like forward and backward propagation happen in float16 - this allows these calculations to be made with a [4X speed up](https://github.com/huggingface/pytorch-pretrained-BERT/issues/149).  

We shall apply BERT to the problem for classifiying toxicity, using apex from Nvidia. We shall compare the impact of hardware by running the model on a V100 and P100 and comparing the speed and accuracy in both cases.   

This script relies heavily on an existing [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967). 
  
*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

In [1]:
import sys, os
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

In [2]:
# Let's activate CUDA for GPU based operations
device=torch.device('cuda')

Change the PATH variable to whereever your `week06/hw` directory is located.  
**For the final run we would like you to have a train_size of at least 1 Million rows, and a valid size of at least 500K rows. When you first run the script, feel free to work with a reduced train and valid size for speed.** 

In [3]:
# In bert we need all inputs to have the same length, we will use the first 220 characters. 
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
# We shall run a single epoch (ie. one pass over the data)
EPOCHS = 2
PATH = '/root/v2/week06/hw' # /root/v2/week06/hw"
DATA_DIR = os.path.join(PATH, "data")
WORK_DIR = os.path.join(PATH, "workingdir")

# Validation and training sizes are here. 
# train_size= 10000 # 1000000 
# valid_size= 5000  # 500000

train_size= 1000000 
valid_size= 500000

This should be the files you downloaded earlier when you ran `download.sh`

In [4]:
os.listdir(DATA_DIR)

['download.sh', 'test.csv', 'train.csv']

We shall install pytorch BERT implementation.   
If you would like to experiment with or view any code (purely optional, and not graded :) ), you can copy the files from the repo https://github.com/huggingface/pytorch-pretrained-BERT  

In [5]:
%%capture
# from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
# from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
# from pytorch_pretrained_bert.modeling import BertModel
# from pytorch_pretrained_bert import BertConfig
from transformers import BertModel, BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import AdamW

We shall now load the model. When you run this, comment out the `capture` command to understand the archecture.

In [6]:
%%capture
# Translate model from tensorflow to pytorch
# BERT_MODEL_PATH = os.path.join(DATA_DIR, 'uncased_L-12_H-768_A-12')
# convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
#                             os.path.join(BERT_MODEL_PATH, 'bert_model.ckpt'),
#                             os.path.join(BERT_MODEL_PATH, 'bert_config.json'), 
#                             os.path.join(WORK_DIR, 'pytorch_model.bin'))

# shutil.copyfile(os.path.join(BERT_MODEL_PATH, 'bert_config.json'), \
#                 os.path.join(WORK_DIR, 'bert_config.json'))
# This is the Bert configuration file
# bert_config = BertConfig(os.path.join(WORK_DIR, 'bert_config.json'))

bert_config = BertConfig()

Bert needs a special formatting of sentences, so we have a sentence start and end token, as well as separators.   
Thanks to this [script](https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming) for a fast convertor of the sentences.

In [7]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

Now we load the BERT tokenizer and convert the sentences.

In [8]:
%%time
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_all = pd.read_csv(os.path.join(DATA_DIR, "train.csv")).sample(train_size+valid_size,random_state=SEED)
print('loaded %d records' % len(train_all))

# Make sure all comment_text values are strings
train_all['comment_text'] = train_all['comment_text'].astype(str) 

sequences = convert_lines(train_all["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_all=train_all.fillna(0)

loaded 1500000 records


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



33724
CPU times: user 34min 13s, sys: 15.7 s, total: 34min 29s
Wall time: 34min 17s


Let us look at how the tokenising works in BERT, see below how it recongizes misspellings - words the model never saw. 

In [9]:
train_all[["comment_text", 'target']].head()

,comment_text,target
458232,It's difficult for many old people to keep up ...,0.000000
272766,She recognized that her tiny-handed husband is...,0.166667
339129,"HPHY76,\nGood for you for thinking out loud, w...",0.000000
773565,And I bet that in the day you expected your Je...,0.500000
476233,Kennedy will add a much needed and scientifica...,0.000000


Lets tokenize some text (I intentionally mispelled some words to check berts subword information handling)

In [10]:
text = 'Hi, I am learning new things in w251 about deep learning the cloud and teh edge.'
tokens = tokenizer.tokenize(text)
' '.join(tokens)

'hi , i am learning new things in w ##25 ##1 about deep learning the cloud and te ##h edge .'

Added start and end token and convert to ids. This is how it is fed into BERT.

In [11]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = tokenizer.convert_tokens_to_ids(tokens)
' '.join(map(str, input_ids))

'101 7632 1010 1045 2572 4083 2047 2477 1999 1059 17788 2487 2055 2784 4083 1996 6112 1998 8915 2232 3341 1012 102'

When BERT converts this sentence to a torch tensor below is shape of the stored tensors.  
We have 12 input tensors, while the sentence tokens has length 23; where are can you see the 23 tokens in the tensors ?... **Feel free to post in slack or discuss in class**

In [12]:
# put input on gpu and make prediction
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()
# bert = BertModel.from_pretrained(WORK_DIR).cuda()
bert_output = bert(torch.tensor([input_ids]).cuda())

print('Sentence tokens {}'.format(tokens))
print('Number of tokens {}'.format(len(tokens)))
print('Tensor shapes : {}'.format([b.cpu().detach().numpy().shape for b in bert_output[0]]))
print('Number of torch tensors : {}'.format(len(bert_output[0])))

Sentence tokens ['[CLS]', 'hi', ',', 'i', 'am', 'learning', 'new', 'things', 'in', 'w', '##25', '##1', 'about', 'deep', 'learning', 'the', 'cloud', 'and', 'te', '##h', 'edge', '.', '[SEP]']
Number of tokens 23
Tensor shapes : [(2,)]
Number of torch tensors : 1


As it is a binary problem, we change our target to [0,1], instead of float.   
We also split the dataset into a training and validation set, 

In [13]:
train_all['target']=(train_all['target']>=0.5).astype(float)
# Training data - sentences
X = sequences[:train_size] 
# Target - the toxicity. 
y = train_all[['target']].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_all[['target']].values[train_size:]

In [14]:
test_df=train_all.tail(valid_size).copy()
train_df=train_all.head(train_size)

**From here on in we would like you to run BERT.**   
**Please do rely on the script available -  [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967) - for at least the first few steps up to training and prediction.**


**1)**   
**Load the training set to a training dataset. For this you need to load the X sequences and y objects to torch tensors**   
**You can use `torch.utils.data.TensorDataset` to input these into a train_dataset.**

In [15]:
# Training data creations
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

**2)**  
**Set your learning rate and batch size; and optionally random seeds if you want reproducable results**   
**Load your pretrained BERT using `BertForSequenceClassification`**   
**Initialise the gradients and place the model on cuda, set up your optimiser and decay parameters**
**Initialise the model with `apex` (we imprted this as `amp`) for mixed precision training**

In [16]:
# Learning rate and batch size
lr=2e-5
batch_size = 32
accumulation_steps=2

In [17]:
# Optional random seeds
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [18]:
# y columns is simply ["target"], so we can hardcode the length at 1
# changed to refer to bert-base-uncased as we do not cache the model in
# the working dir like the Kaggle scripts.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",cache_dir=None,num_labels=1)
# print(model)

In [19]:
# Initialise the gradients and place the model on cuda
model.zero_grad()
model = model.to(device)

In [20]:
# set up your optimiser and decay parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [21]:
# Initialize the model with apex
train = train_dataset  #?!!

#not sure I need this using AdamW vs BertAdam
num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

# the model automatically starts in training mode, so this is redundant
# Also train() returns self, so the model = model.train() just supresses
# the output in Jupyter.  A semicolon at the end of the line would do 
# just as well
# model=model.train()
model.train();

**3)**  
**Start training your model by iterating through batches in a single epoch of the data**

In [22]:
# tqdm_notebook is an iteratable class that provides access to its arguments
# while providing a nice progress bar for Jupyter Notebook

tq = tqdm_notebook(range(EPOCHS))  # this is just 1 for this experiment

for epoch in tq:
    
    # sample the training data into batches randomly taken from the train_dataset
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    
    # Initialize loss and accuracy
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    
    # start another progress bar for the batches coming from the train_loader
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    
    # Iterate through batches
    for i,(x_batch, y_batch) in tk0:
        # predict the outputs from the batch of inputs
        # It seems that the model prediction returns a one member tuple, not a tensor.
        # hence the need for the stray comma after y_pred so it is a tensor not
        # a tuple that makes the loss function barf.
        # 
        # BIG WTF to the writers of the pytorch docs for not specifying the correct
        # return class - where they bother at all. >-\
        y_pred, = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        
        # print(type(y_batch), type(y_pred))
        # print(y_batch, y_pred)
        
        # Calculate the loss for our run
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        
        # with - as statement: "with" accesses context managment (or __enter__() and __exit__() methods) 
        # in the amp.scale_loss(), and saves the result "as" scaled_loss.  Even if there is an error in 
        # the call to scaled_loss.backward(), the with statement executes exit code in the 
        # object, usually to release resources.
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
            
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            # first loop in the epoch uses this because lossf is intialized to None
            lossf = loss.item()
        
        tk0.set_postfix(loss = lossf)
        
        avg_loss += loss.item() / len(train_loader)
        
        # average of accuracy after applying sigmoid activation
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)






/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


**4)**  
**Store your trained model to disk, you will need it if you choose section 8C.**

In [23]:
torch.save(model.state_dict(), "saved_model")


**5)**   
**Now make a prediction for your validation set.**  

In [24]:
%%time
# Following lines are not needed if running the whole notebook. Here in case we want to reload the model
# model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
# model.load_state_dict(torch.load(output_model_file ))

for param in model.parameters():
    param.requires_grad=False
model.eval()

valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    # Same @#$%^% problem with the returned tuple...
    pred, = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy() # save all predictions for AUC scoring


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



CPU times: user 11min 6s, sys: 2min 46s, total: 13min 52s
Wall time: 13min 48s


**6)**  
**In the yuval's kernel he get a metric based on the metric for the jigsaw competition - it is quite complicated. Instead, we would like you to measure the `AUC`, similar to how you did in homework 04. You can compare the results to HW04**  
*A tip, if your score is lower than homework 04 something is wrong....*

In [25]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_val, valid_preds)))

# AUC score of 0.93957 on the smaller dataset is only slightly higher than the 0.93472 from HW4.
# Expecting improvements with the full dataset

AUC score : 0.96692


**7)**  
**Can you show/print the validation sentences predicted with the highest and lowest toxicity ?**

In [27]:
# numpy has added argpartition which makes it easier to retrieve indexes of the top or bottom N, so we can 
max_idx = np.argmax(valid_preds)
min_idx = np.argmin(valid_preds)

val_text = list(train_all["comment_text"])[train_size:]

print(f"Most Toxic: ({val_text[max_idx]}\n\n")
      
print(f"Least Toxic: {val_text[min_idx]}")


Most Toxic: (You are murdering feckless SCUM.  Your mother would slap you for being the piece of shit that you are.  I hope you die in enormous pain from cancer in your asshole that makes you cry from the pain every time you shit you fucking subhuman scum......Fuck you. I would gladly cut your fucking balls off in a minute....go suck cock you piece of shit! Fuck you and your mother you sorry fuckin swill!


Least Toxic: That's a distinct possibility.


## Comparison of V100 and P100 Performance

| Machine | Tokenizer Time | Training Time | Inferrence Time | AUC Score |
| :---: | :---: | :---: | :---: | :---: |
| V100 | 0:34:41 | 1:29:41 | 0:14:14 | 0.96819 |
| P100 | 0:40:58 | 5:49:30 | 0:59:04 | 0.96817 |

The tokenizer runs 100% on CPU, although the V100 runs faster it is only a 17% reduction in tokenizing time.  

For training and inferrence however, the differences in the GPU capabilities in the two VMs is much greater, with the time reductions on the V100 of 74% and 75% respectively.  Accuracy of the models match to 4 decimal places which is as expected given the common random seeding values.

**8)**  
**Pick only one of the below items and complete it. The last two will take a good amount of time (and partial success on them is fine), so proceed with caution on your choice of items :)** 
  
  
**A. Can you train on two epochs ?**

**B. Can you change the learning rate and improve validation score ?**
   
**C. Make a prediction on the test data set with your downloaded model and submit to Kaggle to see where you score on public LB - check out [Abhishek's](https://www.kaggle.com/abhishek) script - https://www.kaggle.com/abhishek/pytorch-bert-inference . Note, you will need to fork Abhisheks kernel, swap out the weights to your downloaded weights and commit the kernel. When finalised and you get the output, there is a button to submit to the competition**  
  
**D. Get BERT running on the tx2 for a sample of the data.** 
  
**E. Finally, and very challenging -- the `BertAdam` optimiser proved to be suboptimal for this task. There is a better optimiser for this dataset in this script [here](https://www.kaggle.com/cristinasierra/pretext-lstm-tuning-v3). Check out the `custom_loss` function. Can you implement it ? It means getting under the hood of the `BertForSequenceClassification` at the source repo and implementing a modified version locally .  `https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/pytorch_pretrained_bert/modeling.py`**